In [2]:
pip install requests
pip install beautifulsoup

SyntaxError: invalid syntax (155332194.py, line 1)

In [5]:
import csv
import requests
from bs4 import BeautifulSoup
import re
import time

def scrape_amazon_products(url, pages_to_scrape=20, max_products=200):
    headers = {
        'User-Agent': 'Your-User-Agent-String-Goes-Here'
    }

    all_products = []
    products_scraped = 0
    
    for page_number in range(1, pages_to_scrape + 1):
        if products_scraped >= max_products:
            break
        
        print(f"Scraping page {page_number}...")
        page_url = url + f"&page={page_number}"
        response = requests.get(page_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            products = soup.find_all('div', {'data-component-type': 's-search-result'})

            for product in products:
                if products_scraped >= max_products:
                    break
                
                product_info = {}

                # Product URL
                product_link = product.find('a', {'class': 'a-link-normal'})
                if product_link:
                    product_info['URL'] = 'https://www.amazon.in' + product_link['href']
                    # Scraping additional details from the product page
                    product_details = scrape_product_details(product_info['URL'])
                    product_info.update(product_details)

                    all_products.append(product_info)
                    products_scraped += 1

            # Amazon might block requests if made too frequently
            time.sleep(2)  # Add a delay of 2 seconds between requests to avoid being blocked

    return all_products

def scrape_product_details(product_url):
    headers = {
        'User-Agent': 'Your-User-Agent-String-Goes-Here'
    }

    product_info = {}
    response = requests.get(product_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Description
        description = soup.find('div', {'id': 'productDescription'})
        if description:
            product_info['Description'] = description.text.strip()

        # ASIN
        asin = soup.find('th', text='ASIN')
        if asin:
            product_info['ASIN'] = asin.find_next_sibling('td').text.strip()

        # Product Description
        product_desc = soup.find('div', {'id': 'feature-bullets'})
        if product_desc:
            product_info['Product_Description'] = product_desc.text.strip()

        # Manufacturer
        manufacturer = soup.find('th', text='Manufacturer')
        if manufacturer:
            product_info['Manufacturer'] = manufacturer.find_next_sibling('td').text.strip()

    return product_info

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['URL', 'Name', 'Price', 'Rating', 'Reviews', 'Description', 'ASIN', 'Product_Description', 'Manufacturer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for product in data:
            writer.writerow(product)

if __name__ == "__main__":
    amazon_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    scraped_data = scrape_amazon_products(amazon_url, pages_to_scrape=20, max_products=200)

    # Save the scraped data to a CSV file
    save_to_csv(scraped_data, 'amazon_products_with_details.csv')

    print("Scraped data saved to amazon_products_with_details.csv")


Scraping page 1...


C:\Users\anubh\AppData\Local\Temp\ipykernel_2508\1315281187.py:64: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  asin = soup.find('th', text='ASIN')
C:\Users\anubh\AppData\Local\Temp\ipykernel_2508\1315281187.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  manufacturer = soup.find('th', text='Manufacturer')


Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraped data saved to amazon_products_with_details.csv


In [9]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_product_data(url):
    product_data = {'URL': url, 'Description': '', 'ASIN': '', 'Product_Description': '', 'Manufacturer': ''}
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Description
            description_tag = soup.find('meta', attrs={'name': 'description'})
            if description_tag:
                product_data['Description'] = description_tag.get('content', '')

            # ASIN
            asin_tag = soup.find('th', text='ASIN')
            if asin_tag:
                asin_value = asin_tag.find_next('td').text.strip()
                if asin_value != '-':
                    product_data['ASIN'] = asin_value

            # Product Description
            product_desc_tag = soup.find('div', {'id': 'productDescription'})
            if product_desc_tag:
                product_data['Product_Description'] = product_desc_tag.text.strip()

            # Manufacturer
            manufacturer_tag = soup.find('a', {'id': 'bylineInfo'})
            if manufacturer_tag:
                product_data['Manufacturer'] = manufacturer_tag.text.strip()
                
    except Exception as e:
        print(f"Error extracting data from {url}: {e}")
    
    return product_data

def process_product_urls(input_csv, output_csv, max_urls=200):
    with open(input_csv, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = ['URL', 'Description', 'ASIN', 'Product_Description', 'Manufacturer']
        
        with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            
            url_count = 0
            for row in reader:
                if url_count >= max_urls:
                    break
                
                product_data = extract_product_data(row['URL'])
                writer.writerow(product_data)
                url_count += 1

if __name__ == "__main__":
    input_csv = 'amazon_products.csv'
    output_csv = 'extracted_product_data.csv'
    process_product_urls(input_csv, output_csv, max_urls=200)
    print(f"Product data extracted and saved to {output_csv}")


C:\Users\anubh\AppData\Local\Temp\ipykernel_2508\859534781.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  asin_tag = soup.find('th', text='ASIN')


Product data extracted and saved to extracted_product_data.csv


In [11]:
import csv

def merge_csv_data(details_csv, products_csv, merged_csv):
    # Read fieldnames from both CSV files
    details_fieldnames = set()
    with open(details_csv, 'r', newline='', encoding='utf-8') as details_file:
        details_reader = csv.DictReader(details_file)
        details_fieldnames.update(details_reader.fieldnames)

    products_fieldnames = set()
    with open(products_csv, 'r', newline='', encoding='utf-8') as products_file:
        products_reader = csv.DictReader(products_file)
        products_fieldnames.update(products_reader.fieldnames)

    # Determine common fieldnames
    common_fieldnames = details_fieldnames.intersection(products_fieldnames)

    # Read data from amazon_products_with_details.csv
    details_data = {}
    with open(details_csv, 'r', newline='', encoding='utf-8') as details_file:
        details_reader = csv.DictReader(details_file)
        for row in details_reader:
            details_data[row['URL']] = row

    # Read data from amazon_products.csv and merge
    merged_data = []
    with open(products_csv, 'r', newline='', encoding='utf-8') as products_file:
        products_reader = csv.DictReader(products_file)
        for row in products_reader:
            url = row['URL']
            if url in details_data:
                merged_row = {field: details_data[url].get(field, '') for field in common_fieldnames}
                merged_row.update(row)
                merged_data.append(merged_row)

    # Write merged data to a new CSV file
    with open(merged_csv, 'w', newline='', encoding='utf-8') as merged_file:
        writer = csv.DictWriter(merged_file, fieldnames=sorted(list(common_fieldnames) + ['Description', 'ASIN', 'Product_Description', 'Manufacturer']))
        writer.writeheader()
        writer.writerows(merged_data)

if __name__ == "__main__":
    details_csv = 'amazon_products_with_details.csv'
    products_csv = 'amazon_products.csv'
    merged_csv = 'merged_data.csv'
    merge_csv_data(details_csv, products_csv, merged_csv)
    print(f"Merged data saved to {merged_csv}")


Merged data saved to merged_data.csv
